# Imports & Prepare Data

In [5]:
from models.ml_sequential import prepare_data, run_all
import tensorflow as tf
# Disable GPU training, comment out to enable
tf.config.set_visible_devices([], 'GPU')
# Imports used across models
from keras.layers import Input, Dense, TimeDistributed, Dropout
from keras.models import Sequential
from keras.regularizers import l2

features = ['female', 'age', 'height', 'mass', 'ta_set', 'rh_set']
features_scaler, output_scaler, X_padded, y_padded, max_len = prepare_data(features)

# Define the input/output shape
input_shape = (None, X_padded.shape[-1])
output_shape = y_padded.shape[-1]

# Train the model with early stopping
from keras.callbacks import EarlyStopping

# Custom log callback
class Callback(tf.keras.callbacks.Callback):
    SHOW_NUMBER = 10
    counter = 0
    epoch = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch = epoch

    def on_epoch_end(self, epoch, logs=None):
        if self.counter == self.SHOW_NUMBER or self.epoch == 1:
            print(f'Epoch: {self.epoch}, loss: {logs["loss"]:.2f}', end='\r')
            # print(f'Epoch: {self.epoch}, loss: {logs["loss"]:.2f}, val loss: {logs["val_loss"]:.2f}', end='\r')
            if self.epoch > 1:
                self.counter = 0
        self.counter += 1

Max sequence length: 600


In [6]:
from tensorflow.keras.layers import GRU, BatchNormalization

# Sizes
model_sizes = [16]

# MODEL NAME
model_name = 'ml_gru_test'

for model_size in model_sizes:
    # Model architecture
    print("Running model size:", model_size)
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(GRU(model_size, return_sequences=True))
    model.add(BatchNormalization())
    model.add(TimeDistributed(Dense(output_shape)))

    # Compile the model
    model.compile(loss='mse', optimizer='adam')

    # Train the model
    # early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_padded, y_padded, validation_split=0, epochs=100, batch_size=32, callbacks=[Callback()], verbose=0)
    print()

    # Run simulations using trained model
    run_all(model, model_name, features, features_scaler, output_scaler, max_len)
    print()

Running model size: 16
Epoch: 90, loss: 0.01
ml_gru_test
Average TRE RMSE: 0.8076143716324512
Average MTSK RMSE: 2.201018757974094

